In [111]:
import pandas as pd
import numpy as np
import re

In [112]:
# Set pandas display options to show up to 100 rows
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [113]:
data = pd.read_csv('data/upper/ALLSLDU_DP05_AL.csv', dtype={'Line Number': str})

In [114]:
# Load in Alabama DEMOGRAPHIC file for all districts

data.rename(columns={
    'GEONAME': 'District',
    'Line Number': 'PROFLN',
    'Title':'TITLE',
    'Estimate': 'PRF_ESTIMATE',
    'Percent Estimate':'PCT_ESTIMATE', 
    'Margin of Error': 'PRF_MG_ERROR',
    'Percent Margin of Error': 'PCT_MG_ERROR'
}, inplace=True)

# 'Title':'TITLE',
#     'TITLE': 'Title',
# make a dictionary of Line Number keys to Title values
title_dict = dict(zip(data['PROFLN'], data['TITLE']))

# convert the dictionary to dataframe and observe 
title_df = pd.DataFrame(list(title_dict.items()), columns=['PROFLN', 'TITLE'])
title_df.head(10)

,PROFLN,TITLE
0,0,ACS DEMOGRAPHIC AND HOUSING ESTIMATES
1,0.5,NaN
2,0.8,SEX AND AGE
3,1,Total population
4,2,Male
5,3,Female
6,4,Sex ratio (males per 100 females)
7,4.3,NaN
8,5,Under 5 years
9,6,5 to 9 years


In [115]:
title_dict.values()

dict_values(['ACS DEMOGRAPHIC AND HOUSING ESTIMATES', nan, 'SEX AND AGE', 'Total population', 'Male', 'Female', 'Sex ratio (males per 100 females)', nan, 'Under 5 years', '5 to 9 years', '10 to 14 years', '15 to 19 years', '20 to 24 years', '25 to 34 years', '35 to 44 years', '45 to 54 years', '55 to 59 years', '60 to 64 years', '65 to 74 years', '75 to 84 years', '85 years and over', nan, 'Median age (years)', nan, 'Under 18 years', '16 years and over', '18 years and over', '21 years and over', '62 years and over', '65 years and over', nan, '18 years and over', 'Male', 'Female', 'Sex ratio (males per 100 females)', nan, '65 years and over', 'Male', 'Female', 'Sex ratio (males per 100 females)', nan, 'RACE', 'Total population', 'One race', 'Two or more races', nan, 'One race', 'White', 'Black or African American', 'American Indian and Alaska Native', 'Cherokee tribal grouping', 'Chippewa tribal grouping', 'Navajo tribal grouping', 'Sioux tribal grouping', 'Asian', 'Asian Indian', 'Chin

In [116]:
title_dict

{'0': 'ACS DEMOGRAPHIC AND HOUSING ESTIMATES',
 '0.5': nan,
 '0.8': 'SEX AND AGE',
 '1': 'Total population',
 '2': 'Male',
 '3': 'Female',
 '4': 'Sex ratio (males per 100 females)',
 '4.3': nan,
 '5': 'Under 5 years',
 '6': '5 to 9 years',
 '7': '10 to 14 years',
 '8': '15 to 19 years',
 '9': '20 to 24 years',
 '10': '25 to 34 years',
 '11': '35 to 44 years',
 '12': '45 to 54 years',
 '13': '55 to 59 years',
 '14': '60 to 64 years',
 '15': '65 to 74 years',
 '16': '75 to 84 years',
 '17': '85 years and over',
 '17.3': nan,
 '18': 'Median age (years)',
 '18.3': nan,
 '19': 'Under 18 years',
 '20': '16 years and over',
 '21': '18 years and over',
 '22': '21 years and over',
 '23': '62 years and over',
 '24': '65 years and over',
 '24.3': nan,
 '25': '18 years and over',
 '26': 'Male',
 '27': 'Female',
 '28': 'Sex ratio (males per 100 females)',
 '28.3': nan,
 '29': '65 years and over',
 '30': 'Male',
 '31': 'Female',
 '32': 'Sex ratio (males per 100 females)',
 '32.3': nan,
 '32.5': 'RAC

In [117]:
## Combining PROFLN numbers which will become new TITLEs

# "Total Population-" Category
for i in range(2, 25):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-' + str(i)

# "Total Population- 18 years and over-" Category
for i in range(25, 29):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-25' + ('' if i == 25 else '-' + str(i))

#"Total Population- 65 years and over-" Category
for i in range(29, 33):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-29' + ('' if i == 29 else '-' + str(i))

# "RACE-" Category
for i in range(33, 37):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-' + str(i)

# "RACE- One race-" Category
for i in range(37, 58):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-36-' + str(i)

# "RACE- Two or more races-" Category
for i in range(58, 63):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-58' + ('' if i == 58 else '-' + str(i))

#"RACE- Race alone or in combination with one or more other races-" Category
for i in range(63, 70):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-62.5-' + str(i)

# 'HISPANIC OR LATINO AND RACE-" Category
for i in range(70, 86):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '69.5-' + str(i)

# 'CITIZEN, VOTING AGE POPULATION' Category
for i in range(87, 90):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '86.5-' + str(i)

In [118]:
# Translate new profln numbers to combo titles
def translate_profln(profln):
    # split the PROFLN value into individual keys
    keys = profln.split('-')

    # translate each key from the title_dict
    titles = [str(title_dict.get(key, '')) for key in keys]

    # concat translated title strings
    translated_title = ' - '.join(title for title in titles if title)

    return translated_title

# replace TITLE values with their new names using function
data['TITLE'] = data['PROFLN'].apply(translate_profln)
data['TITLE'].value_counts()


TITLE
nan                                                                                                              420
RACE - Two or more races                                                                                          70
RACE - One race                                                                                                   70
Total population - 65 years and over                                                                              70
Total population - 18 years and over                                                                              70
RACE - One race - Native Hawaiian                                                                                 35
RACE - Two or more races - Black or African American and American Indian and Alaska Native                        35
RACE - One race - Other Asian                                                                                     35
RACE - Race alone or in combination with one or more other

In [119]:
# remove rows with NaN's, placeholder 'N' for 'Not Large Enough Sample Size', and placeholder '(X)'' rows
invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()

# filter out these rows from the Dataframe
data = data[~invalid_rows]



In [120]:
# remove rows of duplicate information or strong overlap

# List of PROFLN values to remove (many values are repeats from other categories and subcategories)
profln_to_remove = ['1-29', '1-25', '32.5-33', '32.5-36', '32.5-58', 
                    '32.5-62.5-63', '32.5-62.5-64', '32.5-62.5-65', 
                    '32.5-62.5-66', '32.5-62.5-67', '32.5-62.5-68', 
                    '32.5-62.5-69', '69.5-70', '69.5-76', '69.5-77', 
                    '69.5-78', '69.5-79', '69.5-80', '69.5-81', '69.5-82']

# filter the data to observe which rows we are removing
rows_to_remove = data[data['PROFLN'].isin(profln_to_remove)]
print(rows_to_remove['TITLE'])

# Remove these rows from the DataFrame
data = data[~data['PROFLN'].isin(profln_to_remove)]

31                                                Total population - 18 years and over
36                                                Total population - 65 years and over
42                                                             RACE - Total population
46                                                                     RACE - One race
68                                                            RACE - Two or more races
                                             ...                                      
3730                     HISPANIC OR LATINO AND RACE - Black or African American alone
3731             HISPANIC OR LATINO AND RACE - American Indian and Alaska Native alone
3732                                         HISPANIC OR LATINO AND RACE - Asian alone
3733    HISPANIC OR LATINO AND RACE - Native Hawaiian and Other Pacific Islander alone
3734                               HISPANIC OR LATINO AND RACE - Some other race alone
Name: TITLE, Length: 700, dtype: object


In [121]:
# Delete margin of error columns
data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

# Some MEDIAN and MEAN values are not listed in PCT_ESTIMATE column, but are relevant
data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

# Convert all the values to numeric
data['PCT_ESTIMATE'] = data['PCT_ESTIMATE'].str.replace(',', '').astype(float)
data['PRF_ESTIMATE'] = data['PRF_ESTIMATE'].str.replace(',', '').astype(float)

data

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE
3,610U800US01001,"State Senate District 1 (2022), Alabama",1,Total population,144420.0,144420.0
4,610U800US01001,"State Senate District 1 (2022), Alabama",1-2,Total population - Male,70028.0,48.5
5,610U800US01001,"State Senate District 1 (2022), Alabama",1-3,Total population - Female,74392.0,51.5
6,610U800US01001,"State Senate District 1 (2022), Alabama",1-4,Total population - Sex ratio (males per 100 females),94.1,94.1
8,610U800US01001,"State Senate District 1 (2022), Alabama",1-5,Total population - Under 5 years,7209.0,5.0
...,...,...,...,...,...,...
3737,610U800US01035,"State Senate District 35 (2022), Alabama",69.5-85,"HISPANIC OR LATINO AND RACE - Two races excluding Some other race, and Three or more races",2714.0,2.0
3739,610U800US01035,"State Senate District 35 (2022), Alabama",86,Total housing units,58835.0,58835.0
3742,610U800US01035,"State Senate District 35 (2022), Alabama",86.5-87,"CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population",101758.0,101758.0
3743,610U800US01035,"State Senate District 35 (2022), Alabama",86.5-88,"CITIZEN, VOTING AGE POPULATION - Male",48181.0,47.3


In [122]:
## Converting the long district titles to their state and district codes ('AL-01' = 'Alabama District 1')
# State Mapping
states = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

def format_senate_district(district):
    # Regular expression to extract the district number and state from the new format
    match = re.search(r'State Senate District (\d+) \(\d+\), ([A-Za-z\s]+)', district)
    if match:
        district_num, state_name = match.groups()
        district_num = district_num.zfill(2)  # Pad the district number with leading zeros if necessary
        state_abbr = states.get(state_name.strip(), state_name)  # Get the state abbreviation
        return f'{state_abbr}-Sen-{district_num}'  # Format it as required
    return district  # Return the original if no match found


data['Formatted_District'] = data['District'].apply(format_senate_district)

In [123]:
#ensure all districts have the same number of values
data['Formatted_District'].value_counts()


Formatted_District
AL-Sen-01    65
AL-Sen-27    65
AL-Sen-21    65
AL-Sen-22    65
AL-Sen-23    65
AL-Sen-24    65
AL-Sen-25    65
AL-Sen-26    65
AL-Sen-28    65
AL-Sen-19    65
AL-Sen-29    65
AL-Sen-30    65
AL-Sen-31    65
AL-Sen-32    65
AL-Sen-33    65
AL-Sen-34    65
AL-Sen-20    65
AL-Sen-18    65
AL-Sen-02    65
AL-Sen-09    65
AL-Sen-03    65
AL-Sen-04    65
AL-Sen-05    65
AL-Sen-06    65
AL-Sen-07    65
AL-Sen-08    65
AL-Sen-10    65
AL-Sen-17    65
AL-Sen-11    65
AL-Sen-12    65
AL-Sen-13    65
AL-Sen-14    65
AL-Sen-15    65
AL-Sen-16    65
AL-Sen-35    65
Name: count, dtype: int64

In [124]:
# Pivot the dataframe separately into two dataframes with 'PRF_ESTIMATE' (raw numbers) and 'PCT_ESTIMATE' (%'s)
prf_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PRF_ESTIMATE')
pct_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PCT_ESTIMATE')

# Reset the column names after pivot
prf_estimate_df.columns = [f'{col}' for col in prf_estimate_df.columns]
pct_estimate_df.columns = [f'{col}' for col in pct_estimate_df.columns]

# Reset the index
prf_estimate_df.reset_index(inplace=True)
pct_estimate_df.reset_index(inplace=True)

prf_estimate_df

,Formatted_District,GEOID,"CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population","CITIZEN, VOTING AGE POPULATION - Female","CITIZEN, VOTING AGE POPULATION - Male",HISPANIC OR LATINO AND RACE - Cuban,HISPANIC OR LATINO AND RACE - Hispanic or Latino (of any race),HISPANIC OR LATINO AND RACE - Mexican,HISPANIC OR LATINO AND RACE - Other Hispanic or Latino,HISPANIC OR LATINO AND RACE - Puerto Rican,HISPANIC OR LATINO AND RACE - Two or more races,"HISPANIC OR LATINO AND RACE - Two races excluding Some other race, and Three or more races",HISPANIC OR LATINO AND RACE - Two races including Some other race,RACE - One race,RACE - One race - American Indian and Alaska Native,RACE - One race - Asian,RACE - One race - Asian Indian,RACE - One race - Black or African American,RACE - One race - Chamorro,RACE - One race - Chinese,RACE - One race - Filipino,RACE - One race - Japanese,RACE - One race - Korean,RACE - One race - Native Hawaiian,RACE - One race - Native Hawaiian and Other Pacific Islander,RACE - One race - Other Asian,RACE - One race - Other Pacific Islander,RACE - One race - Samoan,RACE - One race - Some other race,RACE - One race - Vietnamese,RACE - One race - White,RACE - Two or more races,RACE - Two or more races - Black or African American and American Indian and Alaska Native,RACE - Two or more races - White and American Indian and Alaska Native,RACE - Two or more races - White and Asian,RACE - Two or more races - White and Black or African American,Total housing units,Total population,Total population - 10 to 14 years,Total population - 15 to 19 years,Total population - 16 years and over,Total population - 18 years and over,Total population - 18 years and over - Female,Total population - 18 years and over - Male,Total population - 18 years and over - Sex ratio (males per 100 females),Total population - 20 to 24 years,Total population - 21 years and over,Total population - 25 to 34 years,Total population - 35 to 44 years,Total population - 45 to 54 years,Total population - 5 to 9 years,Total population - 55 to 59 years,Total population - 60 to 64 years,Total population - 62 years and over,Total population - 65 to 74 years,Total population - 65 years and over,Total population - 65 years and over - Female,Total population - 65 years and over - Male,Total population - 65 years and over - Sex ratio (males per 100 females),Total population - 75 to 84 years,Total population - 85 years and over,Total population - Female,Total population - Male,Total population - Median age (years),Total population - Sex ratio (males per 100 females),Total population - Under 18 years,Total population - Under 5 years
0,AL-Sen-01,610U800US01001,113137.0,58694.0,54443.0,283.0,5335.0,4073.0,682.0,297.0,3642.0,2946.0,696.0,139290.0,691.0,981.0,263.0,11791.0,29.0,251.0,53.0,61.0,40.0,23.0,99.0,56.0,47.0,0.0,1634.0,257.0,124094.0,5130.0,42.0,908.0,491.0,1379.0,66317.0,144420.0,9142.0,9356.0,118470.0,115002.0,59620.0,55382.0,92.9,10351.0,108302.0,17349.0,16733.0,19148.0,7690.0,10672.0,9918.0,32962.0,16147.0,26852.0,14826.0,12026.0,81.1,8383.0,2322.0,74392.0,70028.0,41.4,94.1,29418.0,7209.0
1,AL-Sen-02,610U800US01002,110858.0,55307.0,55551.0,168.0,9178.0,4724.0,2193.0,2093.0,4825.0,4349.0,476.0,143348.0,653.0,5792.0,1107.0,40245.0,19.0,1001.0,337.0,474.0,842.0,0.0,86.0,989.0,64.0,3.0,2331.0,1042.0,94241.0,6542.0,181.0,960.0,1060.0,1937.0,61732.0,149890.0,10856.0,11354.0,120075.0,115617.0,57639.0,57978.0,100.6,11304.0,108551.0,22118.0,20054.0,20925.0,9408.0,10226.0,8673.0,22088.0,10527.0,17665.0,9649.0,8016.0,83.1,5398.0,1740.0,74024.0,75866.0,36.2,102.5,34273.0,7307.0
2,AL-Sen-03,610U800US01003,109617.0,56710.0,52907.0,307.0,12025.0,7599.0,2984.0,1135.0,4168.0,3650.0,518.0,141865.0,639.0,1607.0,880.0,23025.0,22.0,265.0,117.0,0.0,76.0,260.0,282.0,179.0,0.0,0.0,5413.0,90.0,110899.0,6405.0,482.0,1881.0,452.0,673.0,64333.0,148270.0,9493.0,8810.0,117720.0,113730.0,58498.0,55232.0,94.4,8651.0,109364.0,18760.0,19157.0,19683.0,9864.0,9819.0,10654.0,30630.

In [125]:
#LOAD FUNCTION

def load_and_rename_data(csv):
    data = pd.read_csv(csv, dtype={'Line Number': str})
    data.rename(columns={
            'GEONAME': 'District',
            'Line Number': 'PROFLN',
            'Title': 'TITLE',
            'Estimate': 'PRF_ESTIMATE',
            'Percent Estimate':'PCT_ESTIMATE', 
            'Margin of Error': 'PRF_MG_ERROR',
            'Percent Margin of Error': 'PCT_MG_ERROR'
             }, inplace=True)
     # Filter out rows where 'District' mentions "not defined"
    data = data[~data['District'].str.contains("not defined", case=False, na=False)]
    return data

In [126]:
## translate PROFLN number into associated title
def translate_profln(profln, title_dict):
    # split the PROFLN value into keys by '-' for multi-labeled categories
    keys = profln.split('-')
    # translate each key using the title_dict
    titles = [str(title_dict.get(key, '')) for key in keys]
    # concatenate the translated titles
    translated_title = ' - '.join(title for title in titles if title)

    return translated_title

In [127]:
def update_titles_demo(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))
    # Update PROFLN values
    for i in range(2, 25):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-' + str(i)
    for i in range(25, 29):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-25' + ('' if i == 25 else '-' + str(i))
    for i in range(29, 33):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-29' + ('' if i == 29 else '-' + str(i))
    for i in range(33, 37):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-' + str(i)
    for i in range(37, 58):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-36-' + str(i)
    for i in range(58, 63):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-58' + ('' if i == 58 else '-' + str(i))
    for i in range(63, 70):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-62.5-' + str(i)
    for i in range(70, 86):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '69.5-' + str(i)
    for i in range(87, 90):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '86.5-' + str(i)

    # Update TITLE values
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict)) #using translate_profln function

    return data


In [128]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FUNCTION

def clean_up_data_demo(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()
    data = data[~invalid_rows]

    # The rows associated with these index markers (profln) contain duplicated data
    profln_to_remove = ['1-29', '1-25', '32.5-33', '32.5-36', '32.5-58', 
                        '32.5-62.5-63', '32.5-62.5-64', '32.5-62.5-65', 
                        '32.5-62.5-66', '32.5-62.5-67', '32.5-62.5-68', 
                        '32.5-62.5-69', '69.5-70', '69.5-76', '69.5-77', 
                        '69.5-78', '69.5-79', '69.5-80', '69.5-81', '69.5-82']
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually, average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

    # convert to numeric
    data['PCT_ESTIMATE'] = data['PCT_ESTIMATE'].str.replace(',', '').astype(float)
    data['PRF_ESTIMATE'] = data['PRF_ESTIMATE'].str.replace(',', '').astype(float)
    
    return data


In [129]:
# SHORTHAND STATE-DISTRICT FUNCTION

def format_districts(data):
    # dictionary of state names to abbreviations
    states = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
        'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
        'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
        'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
        'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
        'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
        'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    }

    # function to format a single district
    def format_district(district):
        # extract the state name and district number from the district string
        match = re.search(r'State Senate District (\d+) \(\d+\), ([A-Za-z\s]+)', district)
        if match:
            district_num, state_name = match.groups()
            district_num = district_num.zfill(2)  # Pad the district number with leading zeros if necessary
            state_abbr = states.get(state_name.strip(), state_name)  # Get the state abbreviation
            return f'{state_abbr}-Sen-{district_num}'  # Format it as required
        else:
            return district

    # Apply the function to the District column
    data['Formatted_District'] = data['District'].apply(format_district)

    return data


In [130]:
## TRANSPOSE DATAFRAME FUNCTION
def reorient_dfs(data):
    # Pivot the dataframe separately for 'PRF_ESTIMATE' and 'PCT_ESTIMATE'
    prf_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PRF_ESTIMATE')
    pct_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PCT_ESTIMATE')

    # Reset the column names after pivot
    prf_estimate_df.columns = [f'{col}' for col in prf_estimate_df.columns]
    pct_estimate_df.columns = [f'{col}' for col in pct_estimate_df.columns]

    # Reset the index
    prf_estimate_df.reset_index(inplace=True)
    pct_estimate_df.reset_index(inplace=True)

    return prf_estimate_df, pct_estimate_df


# Process all 50 states demographic characteristics

In [131]:
## PROCESS ALL 50 STATES FUNCTION
def process_demographics(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_demo(data)
        data = clean_up_data_demo(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfdemo_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctdemo_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfdemo_all, df_pctdemo_all


In [132]:
file_state_list = [
 ('data/upper/ALLSLDU_DP05_AK.csv', 'AK'),('data/upper/ALLSLDU_DP05_AL.csv', 'AL'),('data/upper/ALLSLDU_DP05_AR.csv', 'AR'),
 ('data/upper/ALLSLDU_DP05_AZ.csv', 'AZ'),('data/upper/ALLSLDU_DP05_CA.csv', 'CA'),('data/upper/ALLSLDU_DP05_CO.csv', 'CO'),
 ('data/upper/ALLSLDU_DP05_CT.csv', 'CT'),('data/upper/ALLSLDU_DP05_DE.csv', 'DE'),('data/upper/ALLSLDU_DP05_FL.csv', 'FL'),
 ('data/upper/ALLSLDU_DP05_GA.csv', 'GA'),('data/upper/ALLSLDU_DP05_HI.csv', 'HI'),('data/upper/ALLSLDU_DP05_IA.csv', 'IA'),
 ('data/upper/ALLSLDU_DP05_ID.csv', 'ID'),('data/upper/ALLSLDU_DP05_IL.csv', 'IL'),('data/upper/ALLSLDU_DP05_IN.csv', 'IN'),
 ('data/upper/ALLSLDU_DP05_KS.csv', 'KS'),('data/upper/ALLSLDU_DP05_KY.csv', 'KY'),('data/upper/ALLSLDU_DP05_LA.csv', 'LA'),
 ('data/upper/ALLSLDU_DP05_MA.csv', 'MA'),('data/upper/ALLSLDU_DP05_MD.csv', 'MD'),('data/upper/ALLSLDU_DP05_ME.csv', 'ME'),
 ('data/upper/ALLSLDU_DP05_MI.csv', 'MI'),('data/upper/ALLSLDU_DP05_MN.csv', 'MN'),('data/upper/ALLSLDU_DP05_MO.csv', 'MO'),
 ('data/upper/ALLSLDU_DP05_MS.csv', 'MS'),('data/upper/ALLSLDU_DP05_MT.csv', 'MT'),('data/upper/ALLSLDU_DP05_NC.csv', 'NC'),
 ('data/upper/ALLSLDU_DP05_ND.csv', 'ND'),('data/upper/ALLSLDU_DP05_NE.csv', 'NE'),('data/upper/ALLSLDU_DP05_NH.csv', 'NH'),
 ('data/upper/ALLSLDU_DP05_NJ.csv', 'NJ'),('data/upper/ALLSLDU_DP05_NM.csv', 'NM'),('data/upper/ALLSLDU_DP05_NV.csv', 'NV'),
 ('data/upper/ALLSLDU_DP05_NY.csv', 'NY'),('data/upper/ALLSLDU_DP05_OH.csv', 'OH'),('data/upper/ALLSLDU_DP05_OK.csv', 'OK'),
 ('data/upper/ALLSLDU_DP05_OR.csv', 'OR'),('data/upper/ALLSLDU_DP05_PA.csv', 'PA'),('data/upper/ALLSLDU_DP05_RI.csv', 'RI'),
 ('data/upper/ALLSLDU_DP05_SC.csv', 'SC'),('data/upper/ALLSLDU_DP05_SD.csv', 'SD'),('data/upper/ALLSLDU_DP05_TN.csv', 'TN'),
 ('data/upper/ALLSLDU_DP05_TX.csv', 'TX'),('data/upper/ALLSLDU_DP05_UT.csv', 'UT'),('data/upper/ALLSLDU_DP05_VA.csv', 'VA'),
 ('data/upper/ALLSLDU_DP05_VT.csv', 'VT'),('data/upper/ALLSLDU_DP05_WA.csv', 'WA'),('data/upper/ALLSLDU_DP05_WI.csv', 'WI'),
 ('data/upper/ALLSLDU_DP05_WV.csv', 'WV'),('data/upper/ALLSLDU_DP05_WY.csv', 'WY')
]


df_prfdemo_all, df_pctdemo_all = process_demographics(file_state_list)


In [133]:
df_prfdemo_all.columns

Index(['Formatted_District', 'GEOID',
       'CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population',
       'CITIZEN, VOTING AGE POPULATION - Female',
       'CITIZEN, VOTING AGE POPULATION - Male',
       'HISPANIC OR LATINO AND RACE - Cuban',
       'HISPANIC OR LATINO AND RACE - Hispanic or Latino (of any race)',
       'HISPANIC OR LATINO AND RACE - Mexican',
       'HISPANIC OR LATINO AND RACE - Other Hispanic or Latino',
       'HISPANIC OR LATINO AND RACE - Puerto Rican',
       'HISPANIC OR LATINO AND RACE - Two or more races',
       'HISPANIC OR LATINO AND RACE - Two races excluding Some other race, and Three or more races',
       'HISPANIC OR LATINO AND RACE - Two races including Some other race',
       'RACE - One race',
       'RACE - One race - American Indian and Alaska Native',
       'RACE - One race - Asian', 'RACE - One race - Asian Indian',
       'RACE - One race - Black or African American',
       'RACE - One race - Chamorro', 'RACE - One race - Chi

In [134]:
df_pctdemo_all.shape

(1942, 67)

# Housing Characteristics

In [135]:
## LOAD_AND_RENAME_DATA FUNCTION REMAINS THE SAME
AL_housing_df = load_and_rename_data('data/upper/ALLSLDU_DP04_AL.csv')

In [136]:
#Profln values and their corresponding TITLE/associations have changed. Revamping title formatting
def update_titles_housing(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))
    
    #Update PROFLN numbers by category
    for i in range(1, 6): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-' + str(i) # "HOUSING OCCUPANCY-" Category
    for i in range(6, 16): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '5.5-' + str(i) # "UNITS IN STRUCTURE-" Category
    for i in range(16, 27): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '15.5-' + str(i) # "YEAR STRUCTURE BUILT-" Category
    for i in range(27, 38): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '26.5-' + str(i) # "ROOMS-" Category
    for i in range(39, 45): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '37.5-' + str(i) # "BEDROOMS-" Category
    for i in range(45, 57): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '44.5-' + str(i) # "HOUSING TENURE-" Category
    for i in range(57, 62): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '56.5-' + str(i) # "VEHICLES AVAILABLE-" Category
    for i in range(62, 72): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '61.5-' + str(i) # "HOUSE HEATING FUEL-" Category
    for i in range(72, 76): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '71.5-' + str(i) # "SELECTED CHARACTERISTICS-" Category
    for i in range(76, 80): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '75.5-' + str(i) # "OCCUPANTS PER ROOM-" Category
    for i in range(80, 90): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '79.5-' + str(i)  # "VALUE-" Category
    for i in range(90, 93): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '89.5-' + str(i)   # "MORTGAGE STATUS-" Category
    for i in range(126, 136):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '125.5-' + str(i) # "GROSS RENT-" Category
        
    ###LONG TITLES BELOW###
    
    # "SELECTED MONTHLY OWNER COSTS (SMOC)- Housing units with a mortgage- " Category
    for i in range(93, 102):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '92.5-93' + ('' if i == 93 else '-' + str(i))
    # "SELECTED MONTHLY OWNER COSTS (SMOC)- Housing units without a mortgage- " Category
    for i in range(102, 110):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '92.5-102' + ('' if i == 102 else '-' + str(i))  
    # "SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)- 
    # Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)-" Category
    for i in range(110, 116):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '109.5-110' + ('' if i == 110 else '-' + str(i))  
    # "SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)-
    # Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)-" Category
    for i in range(117, 125):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '109.5-117' + ('' if i == 117 else '-' + str(i))
    #"GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)-" Category
    for i in range(136, 143):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '135.5-' + str(i)
        
    # Translate TITLE values using previously defined 'translate_profln' function
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))

    return data

In [137]:
AL_housing_df = update_titles_housing(AL_housing_df)

In [138]:
import pandas as pd

# Temporary settings to display more data
with pd.option_context('display.max_rows', None):
    print(AL_housing_df['TITLE'].unique())


['SELECTED HOUSING CHARACTERISTICS' 'nan' 'HOUSING OCCUPANCY'
 'HOUSING OCCUPANCY - Total housing units'
 'HOUSING OCCUPANCY - Occupied housing units'
 'HOUSING OCCUPANCY - Vacant housing units'
 'HOUSING OCCUPANCY - Homeowner vacancy rate'
 'HOUSING OCCUPANCY - Rental vacancy rate' 'UNITS IN STRUCTURE'
 'UNITS IN STRUCTURE - Total housing units'
 'UNITS IN STRUCTURE - 1-unit, detached'
 'UNITS IN STRUCTURE - 1-unit, attached' 'UNITS IN STRUCTURE - 2 units'
 'UNITS IN STRUCTURE - 3 or 4 units' 'UNITS IN STRUCTURE - 5 to 9 units'
 'UNITS IN STRUCTURE - 10 to 19 units'
 'UNITS IN STRUCTURE - 20 or more units'
 'UNITS IN STRUCTURE - Mobile home'
 'UNITS IN STRUCTURE - Boat, RV, van, etc.' 'YEAR STRUCTURE BUILT'
 'YEAR STRUCTURE BUILT - Total housing units'
 'YEAR STRUCTURE BUILT - Built 2020 or later'
 'YEAR STRUCTURE BUILT - Built 2010 to 2019'
 'YEAR STRUCTURE BUILT - Built 2000 to 2009'
 'YEAR STRUCTURE BUILT - Built 1990 to 1999'
 'YEAR STRUCTURE BUILT - Built 1980 to 1989'
 'YEAR STR

In [139]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FROM HOUSING DATASET FUNCTION

def clean_up_data_housing(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['5.5-6','15.5-16', '26.5-27', '38',                        # Duplicated Total Housing Units
                        '44.5-45','44.5-50', '56.5-57', '61.5-62','71.5-72', '75.5-76', #Dup. Occupied HU
                        '79.5-80', '89.5-90', '92.5-93','109.5-110','92.5-102', '109.5-117', #Mortgage/No Mortgage
                        '135.5-136']                                                       #Dup. HU Renting
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

    # Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data



In [140]:
AL_housing_df = clean_up_data_housing(AL_housing_df)

In [141]:
format_districts(AL_housing_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
3,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1,HOUSING OCCUPANCY - Total housing units,66317.0,66317.0,AL-Sen-01
4,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-2,HOUSING OCCUPANCY - Occupied housing units,57820.0,87.2,AL-Sen-01
5,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-3,HOUSING OCCUPANCY - Vacant housing units,8497.0,12.8,AL-Sen-01
7,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-4,HOUSING OCCUPANCY - Homeowner vacancy rate,1.1,1.1,AL-Sen-01
8,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-5,HOUSING OCCUPANCY - Rental vacancy rate,3.8,3.8,AL-Sen-01
...,...,...,...,...,...,...,...
6503,610U800US01035,"State Senate District 35 (2022), Alabama",135.5-138,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 15.0 to 19.9 percent,1714.0,13.3,AL-Sen-35
6504,610U800US01035,"State Senate District 35 (2022), Alabama",135.5-139,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 20.0 to 24.9 percent,1367.0,10.6,AL-Sen-35
6505,610U800US01035,"State Senate District 35 (2022), Alabama",135.5-140,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 25.0 to 29.9 percent,1433.0,11.1,AL-Sen-35
6506,610U800US01035,"State Senate District 35 (2022), Alabama",135.5-141,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 30.0 to 34.9 percent,1120.0,8.7,AL-Sen-35


In [142]:
AL_housing_prf, AL_housing_pct = reorient_dfs(AL_housing_df)

In [143]:
AL_housing_pct

,Formatted_District,GEOID,BEDROOMS - 1 bedroom,BEDROOMS - 2 bedrooms,BEDROOMS - 3 bedrooms,BEDROOMS - 4 bedrooms,BEDROOMS - 5 or more bedrooms,BEDROOMS - No bedroom,"GROSS RENT - $1,000 to $1,499","GROSS RENT - $1,500 to $1,999","GROSS RENT - $2,000 to $2,499","GROSS RENT - $2,500 to $2,999","GROSS RENT - $3,000 or more",GROSS RENT - $500 to $999,GROSS RENT - Less than $500,GROSS RENT - Median (dollars),GROSS RENT - No rent paid,GROSS RENT - Occupied units paying rent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 15.0 to 19.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 20.0 to 24.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 25.0 to 29.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 30.0 to 34.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 35.0 percent or more,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - Less than 15.0 percent,"HOUSE HEATING FUEL - Bottled, tank, or LP gas",HOUSE HEATING FUEL - Coal or coke,HOUSE HEATING FUEL - Electricity,"HOUSE HEATING FUEL - Fuel oil, kerosene, etc.",HOUSE HEATING FUEL - No fuel used,HOUSE HEATING FUEL - Other fuel,HOUSE HEATING FUEL - Solar energy,HOUSE HEATING FUEL - Utility gas,HOUSE HEATING FUEL - Wood,HOUSING OCCUPANCY - Homeowner vacancy rate,HOUSING OCCUPANCY - Occupied housing units,HOUSING OCCUPANCY - Rental vacancy rate,HOUSING OCCUPANCY - Total housing units,HOUSING OCCUPANCY - Vacant housing units,HOUSING TENURE - Average household size of owner-occupied unit,HOUSING TENURE - Average household size of renter-occupied unit,HOUSING TENURE - Moved in 1989 and earlier,HOUSING TENURE - Moved in 1990 to 1999,HOUSING TENURE - Moved in 2000 to 2009,HOUSING TENURE - Moved in 2010 to 2014,HOUSING TENURE - Moved in 2015 to 2018,HOUSING TENURE - Moved in 2019 or later,HOUSING TENURE - Owner-occupied,HOUSING TENURE - Renter-occupied,MORTGAGE STATUS - Housing units with a mortgage,MORTGAGE STATUS - Housing units without a mortgage,OCCUPANTS PER ROOM - 1.00 or less,OCCUPANTS PER ROOM - 1.01 to 1.50,OCCUPANTS PER ROOM - 1.51 or more,ROOMS - 1 room,ROOMS - 2 rooms,ROOMS - 3 rooms,ROOMS - 4 rooms,ROOMS - 5 rooms,ROOMS - 6 rooms,ROOMS - 7 rooms,ROOMS - 8 rooms,ROOMS - 9 rooms or more,ROOMS - Median rooms,SELECTED CHARACTERISTICS - Lacking complete kitchen facilities,SELECTED CHARACTERISTICS - Lacking complete plumbing facilities,SELECTED CHARACTERISTICS - No telephone service available,"SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $1,000 to $1,499","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $1,500 to $1,999","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $2,000 to $2,499","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $2,500 to $2,999","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $3,000 or more",SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $500 to $999,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - Less than $500,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - Median (dollars),"SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $1,000 or more",SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $250 to $399,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $400 to $599,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $600 to $799,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $800 to $999,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - Less than $250,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - Median (dollars),SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI) - Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) - 10.0 to 14.9 percent,SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE O

In [144]:
## PROCESS ALL 50 STATES HOUSING FUNCTION
def process_housing(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_housing(data)
        data = clean_up_data_housing(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfhousing_all = pd.concat(prf_dfs, ignore_index=True)
    df_pcthousing_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfhousing_all, df_pcthousing_all


In [145]:
file_state_list = [
 ('data/upper/ALLSLDU_DP04_AK.csv', 'AK'),('data/upper/ALLSLDU_DP04_AL.csv', 'AL'),('data/upper/ALLSLDU_DP04_AR.csv', 'AR'),
 ('data/upper/ALLSLDU_DP04_AZ.csv', 'AZ'),('data/upper/ALLSLDU_DP04_CA.csv', 'CA'),('data/upper/ALLSLDU_DP04_CO.csv', 'CO'),
 ('data/upper/ALLSLDU_DP04_CT.csv', 'CT'),('data/upper/ALLSLDU_DP04_DE.csv', 'DE'),('data/upper/ALLSLDU_DP04_FL.csv', 'FL'),
 ('data/upper/ALLSLDU_DP04_GA.csv', 'GA'),('data/upper/ALLSLDU_DP04_HI.csv', 'HI'),('data/upper/ALLSLDU_DP04_IA.csv', 'IA'),
 ('data/upper/ALLSLDU_DP04_ID.csv', 'ID'),('data/upper/ALLSLDU_DP04_IL.csv', 'IL'),('data/upper/ALLSLDU_DP04_IN.csv', 'IN'),
 ('data/upper/ALLSLDU_DP04_KS.csv', 'KS'),('data/upper/ALLSLDU_DP04_KY.csv', 'KY'),('data/upper/ALLSLDU_DP04_LA.csv', 'LA'),
 ('data/upper/ALLSLDU_DP04_MA.csv', 'MA'),('data/upper/ALLSLDU_DP04_MD.csv', 'MD'),('data/upper/ALLSLDU_DP04_ME.csv', 'ME'),
 ('data/upper/ALLSLDU_DP04_MI.csv', 'MI'),('data/upper/ALLSLDU_DP04_MN.csv', 'MN'),('data/upper/ALLSLDU_DP04_MO.csv', 'MO'),
 ('data/upper/ALLSLDU_DP04_MS.csv', 'MS'),('data/upper/ALLSLDU_DP04_MT.csv', 'MT'),('data/upper/ALLSLDU_DP04_NC.csv', 'NC'),
 ('data/upper/ALLSLDU_DP04_ND.csv', 'ND'),('data/upper/ALLSLDU_DP04_NE.csv', 'NE'),('data/upper/ALLSLDU_DP04_NH.csv', 'NH'),
 ('data/upper/ALLSLDU_DP04_NJ.csv', 'NJ'),('data/upper/ALLSLDU_DP04_NM.csv', 'NM'),('data/upper/ALLSLDU_DP04_NV.csv', 'NV'),
 ('data/upper/ALLSLDU_DP04_NY.csv', 'NY'),('data/upper/ALLSLDU_DP04_OH.csv', 'OH'),('data/upper/ALLSLDU_DP04_OK.csv', 'OK'),
 ('data/upper/ALLSLDU_DP04_OR.csv', 'OR'),('data/upper/ALLSLDU_DP04_PA.csv', 'PA'),('data/upper/ALLSLDU_DP04_RI.csv', 'RI'),
 ('data/upper/ALLSLDU_DP04_SC.csv', 'SC'),('data/upper/ALLSLDU_DP04_SD.csv', 'SD'),('data/upper/ALLSLDU_DP04_TN.csv', 'TN'),
 ('data/upper/ALLSLDU_DP04_TX.csv', 'TX'),('data/upper/ALLSLDU_DP04_UT.csv', 'UT'),('data/upper/ALLSLDU_DP04_VA.csv', 'VA'),
 ('data/upper/ALLSLDU_DP04_VT.csv', 'VT'),('data/upper/ALLSLDU_DP04_WA.csv', 'WA'),('data/upper/ALLSLDU_DP04_WI.csv', 'WI'),
 ('data/upper/ALLSLDU_DP04_WV.csv', 'WV'),('data/upper/ALLSLDU_DP04_WY.csv', 'WY')
]


df_prfhousing_all, df_pcthousing_all = process_housing(file_state_list)


In [146]:
df_prfhousing_all.columns

Index(['Formatted_District', 'GEOID', 'BEDROOMS - 1 bedroom',
       'BEDROOMS - 2 bedrooms', 'BEDROOMS - 3 bedrooms',
       'BEDROOMS - 4 bedrooms', 'BEDROOMS - 5 or more bedrooms',
       'BEDROOMS - No bedroom', 'GROSS RENT - $1,000 to $1,499',
       'GROSS RENT - $1,500 to $1,999',
       ...
       'YEAR STRUCTURE BUILT - Built 1939 or earlier',
       'YEAR STRUCTURE BUILT - Built 1940 to 1949',
       'YEAR STRUCTURE BUILT - Built 1950 to 1959',
       'YEAR STRUCTURE BUILT - Built 1960 to 1969',
       'YEAR STRUCTURE BUILT - Built 1970 to 1979',
       'YEAR STRUCTURE BUILT - Built 1980 to 1989',
       'YEAR STRUCTURE BUILT - Built 1990 to 1999',
       'YEAR STRUCTURE BUILT - Built 2000 to 2009',
       'YEAR STRUCTURE BUILT - Built 2010 to 2019',
       'YEAR STRUCTURE BUILT - Built 2020 or later'],
      dtype='object', length=125)

In [147]:
df_prfhousing_all

,Formatted_District,GEOID,BEDROOMS - 1 bedroom,BEDROOMS - 2 bedrooms,BEDROOMS - 3 bedrooms,BEDROOMS - 4 bedrooms,BEDROOMS - 5 or more bedrooms,BEDROOMS - No bedroom,"GROSS RENT - $1,000 to $1,499","GROSS RENT - $1,500 to $1,999","GROSS RENT - $2,000 to $2,499","GROSS RENT - $2,500 to $2,999","GROSS RENT - $3,000 or more",GROSS RENT - $500 to $999,GROSS RENT - Less than $500,GROSS RENT - Median (dollars),GROSS RENT - No rent paid,GROSS RENT - Occupied units paying rent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 15.0 to 19.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 20.0 to 24.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 25.0 to 29.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 30.0 to 34.9 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 35.0 percent or more,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - Less than 15.0 percent,"HOUSE HEATING FUEL - Bottled, tank, or LP gas",HOUSE HEATING FUEL - Coal or coke,HOUSE HEATING FUEL - Electricity,"HOUSE HEATING FUEL - Fuel oil, kerosene, etc.",HOUSE HEATING FUEL - No fuel used,HOUSE HEATING FUEL - Other fuel,HOUSE HEATING FUEL - Solar energy,HOUSE HEATING FUEL - Utility gas,HOUSE HEATING FUEL - Wood,HOUSING OCCUPANCY - Homeowner vacancy rate,HOUSING OCCUPANCY - Occupied housing units,HOUSING OCCUPANCY - Rental vacancy rate,HOUSING OCCUPANCY - Total housing units,HOUSING OCCUPANCY - Vacant housing units,HOUSING TENURE - Average household size of owner-occupied unit,HOUSING TENURE - Average household size of renter-occupied unit,HOUSING TENURE - Moved in 1989 and earlier,HOUSING TENURE - Moved in 1990 to 1999,HOUSING TENURE - Moved in 2000 to 2009,HOUSING TENURE - Moved in 2010 to 2014,HOUSING TENURE - Moved in 2015 to 2018,HOUSING TENURE - Moved in 2019 or later,HOUSING TENURE - Owner-occupied,HOUSING TENURE - Renter-occupied,MORTGAGE STATUS - Housing units with a mortgage,MORTGAGE STATUS - Housing units without a mortgage,OCCUPANTS PER ROOM - 1.00 or less,OCCUPANTS PER ROOM - 1.01 to 1.50,OCCUPANTS PER ROOM - 1.51 or more,ROOMS - 1 room,ROOMS - 2 rooms,ROOMS - 3 rooms,ROOMS - 4 rooms,ROOMS - 5 rooms,ROOMS - 6 rooms,ROOMS - 7 rooms,ROOMS - 8 rooms,ROOMS - 9 rooms or more,ROOMS - Median rooms,SELECTED CHARACTERISTICS - Lacking complete kitchen facilities,SELECTED CHARACTERISTICS - Lacking complete plumbing facilities,SELECTED CHARACTERISTICS - No telephone service available,"SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $1,000 to $1,499","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $1,500 to $1,999","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $2,000 to $2,499","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $2,500 to $2,999","SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $3,000 or more",SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - $500 to $999,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - Less than $500,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units with a mortgage - Median (dollars),"SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $1,000 or more",SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $250 to $399,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $400 to $599,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $600 to $799,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - $800 to $999,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - Less than $250,SELECTED MONTHLY OWNER COSTS (SMOC) - Housing units without a mortgage - Median (dollars),SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI) - Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed) - 10.0 to 14.9 percent,SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE O

# SOCIAL CHARACTERISTICS

In [148]:
## LOAD_AND_RENAME_DATA FUNCTION REMAINS THE SAME
AL_social_df = load_and_rename_data('data/upper/ALLSLDU_DP02_AL.csv')

In [149]:
def update_titles_social(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))

    ####### DICTIONARIES FOR PROFLN NUMBERS THAT NEED A DIRECT MAPPING#######
    # "HOUSEHOLDS BY TYPE" category
    households_by_type_dict = {
        '2': '1-2', '3': '1-2-3', '4': '1-4', '5': '1-4-5',
        '6': '1-6', '7': '1-6-7', '8': '1-6-8', '9': '1-6-9',
        '10': '1-10', '11': '1-10-11', '12': '1-10-12', '13': '1-10-13'
    }
    # "FERTILITY" category
    fertility_dict = {
        '37': '36.5-37', '38': '36.5-38', '39': '36.5-37-39', 
        '40': '36.5-37-40', '41': '36.5-37-41', '42': '36.5-37-42', 
        '43': '36.5-37-43'
    }
    # "GRANDPARENTS" category
    grandparents_dict = {
        '44': '43.5-44', '45': '43.5-44-45', '45.9': '45-45.9',
        '46': '45-45.9-46', '47': '45-45.9-47', '48': '45-45.9-48',
        '49': '45-45.9-49', '51': '50-51', '52': '50-52'
    }
    # "VETERAN STATUS" category
    veteran_status_dict = {
        '69': '68.5-69', '70': '68.5-70'
    }
    # "PLACE OF BIRTH" category
    place_of_birth_dict = {
        '88': '87.5-88', '89': '87.5-89', '90': '87.5-89-90', 
        '91': '87.5-89-91', '92': '87.5-89-92', '93': '87.5-89-93', 
        '94': '87.5-94'
    }
    # "U.S. CITIZENSHIP STATUS" category
    citizenship_status_dict = {
        '96': '95-96', '97': '95-97'
    }
    # "YEAR OF ENTRY" category
    year_of_entry_dict = {
        '99': '98-99', '100': '98-99-100', '101': '98-99-101',
        '102': '98-102', '103': '98-102-103', '104': '98-102-104'
    }
    # "LANGUAGE SPOKEN AT HOME" category
    language_spoken_dict = {
        '112': '111.5-112', '113': '111.5-113', '114': '111.5-114',
        '115': '111.5-114-115', '116': '111.5-116', '117': '111.5-114-117',
        '118': '111.5-118', '119': '111.5-118-119', '120': '111.5-120',
        '121': '111.5-120-121', '122': '111.5-114-122', '123': '111.5-114-122-123'
    }
    # "COMPUTERS AND INTERNET USE" category
    computers_and_internet_use_dict = {
        '152': '151.5-152', 
        '153': '151.5-152-153', 
        '154': '151.5-152-154'
    }
    

    ###### PROFLN NUMBERS THAT FOLLOW A SIMPLE PATTERN OVER A LONG RANGE######
    # "RELATIONSHIP" category
    for i in range(19, 25):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '18-' + str(i)
    # "MARITAL STATUS" category
    for i in range(26, 31):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '25-' + str(i)
    for i in range(32, 37):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '31-' + str(i)
    # "SCHOOL ENROLLMENT" category
    for i in range(54, 59):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '53-' + str(i)
    # "EDUCATIONAL ATTAINMENT" category
    for i in range(59, 69):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '58.5-59' + ('' if i == 59 else '-' + str(i))
    # "DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION" category
    for i in range(72, 79):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '71-' + str(i)
    # "RESIDENCE 1 YEAR AGO" category
    for i in range(80, 88):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '78.5-' + str(i)
    # "WORLD REGION OF BIRTH OF FOREIGN BORN" category
    for i in range(105, 112):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '104.5-' + str(i)
    # "ANCESTRY" category
    for i in range(125, 152):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '123.5-' + str(i)



    ###### DIRECT MAPPING #####
    for old, new in households_by_type_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in fertility_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in grandparents_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in veteran_status_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in place_of_birth_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in citizenship_status_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in year_of_entry_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in language_spoken_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in computers_and_internet_use_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new

    # Translate TITLE values using previously defined 'translate_profln' function
    data['PROFLN'] = data['PROFLN'].astype(str)
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))



    return data


In [150]:
AL_social_df = update_titles_social(AL_social_df)

In [151]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FROM SOCIAL DATASET FUNCTION

def clean_up_data_social(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = (data['PRF_ESTIMATE'].isin(['N', '(X)']) | 
                data['PCT_ESTIMATE'].isin(['N', '(X)']) |
                data['PRF_ESTIMATE'].isna())
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['50','95','98-102', 
                        '1','152','124']  # Duplicated Grandparents, Foreign born, Total households, total pop               
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']
    # Replace '-' in PCT_ESTIMATE with 0 (inspection of data shows at or close to zero but not enough samples)
    data.loc[data['PCT_ESTIMATE'] == '-', 'PCT_ESTIMATE'] = '0'

    # Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data

In [152]:
AL_social_df = clean_up_data_social(AL_social_df)

In [153]:
format_districts(AL_social_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
4,610U800US01001,"State Senate District 1 (2022), Alabama",1-2,Total households - Married-couple household,29923.0,51.8,AL-Sen-01
5,610U800US01001,"State Senate District 1 (2022), Alabama",1-2-3,Total households - Married-couple household - With children of the householder under 18 years,10418.0,18.0,AL-Sen-01
6,610U800US01001,"State Senate District 1 (2022), Alabama",1-4,Total households - Cohabiting couple household,2547.0,4.4,AL-Sen-01
7,610U800US01001,"State Senate District 1 (2022), Alabama",1-4-5,Total households - Cohabiting couple household - With children of the householder under 18 years,801.0,1.4,AL-Sen-01
8,610U800US01001,"State Senate District 1 (2022), Alabama",1-6,"Total households - Male householder, no spouse/partner present",9144.0,15.8,AL-Sen-01
...,...,...,...,...,...,...,...
6993,610U800US01035,"State Senate District 35 (2022), Alabama",123.5-150,ANCESTRY - Welsh,1168.0,0.9,AL-Sen-35
6994,610U800US01035,"State Senate District 35 (2022), Alabama",123.5-151,ANCESTRY - West Indian (excluding Hispanic origin groups),223.0,0.2,AL-Sen-35
6997,610U800US01035,"State Senate District 35 (2022), Alabama",151.5-152,COMPUTERS AND INTERNET USE - Total households,51317.0,51317.0,AL-Sen-35
6998,610U800US01035,"State Senate District 35 (2022), Alabama",151.5-152-153,COMPUTERS AND INTERNET USE - Total households - With a computer,47594.0,92.7,AL-Sen-35


In [154]:
AL_social_prf, AL_social_pct = reorient_dfs(AL_social_df)

In [155]:
AL_social_prf

,Formatted_District,GEOID,ANCESTRY - American,ANCESTRY - Arab,ANCESTRY - Czech,ANCESTRY - Danish,ANCESTRY - Dutch,ANCESTRY - English,ANCESTRY - French (except Basque),ANCESTRY - French Canadian,ANCESTRY - German,ANCESTRY - Greek,ANCESTRY - Hungarian,ANCESTRY - Irish,ANCESTRY - Italian,ANCESTRY - Lithuanian,ANCESTRY - Norwegian,ANCESTRY - Polish,ANCESTRY - Portuguese,ANCESTRY - Russian,ANCESTRY - Scotch-Irish,ANCESTRY - Scottish,ANCESTRY - Slovak,ANCESTRY - Subsaharan African,ANCESTRY - Swedish,ANCESTRY - Swiss,ANCESTRY - Ukrainian,ANCESTRY - Welsh,ANCESTRY - West Indian (excluding Hispanic origin groups),COMPUTERS AND INTERNET USE - Total households,COMPUTERS AND INTERNET USE - Total households - With a broadband Internet subscription,COMPUTERS AND INTERNET USE - Total households - With a computer,EDUCATIONAL ATTAINMENT - Population 25 years and over,"EDUCATIONAL ATTAINMENT - Population 25 years and over - 9th to 12th grade, no diploma",EDUCATIONAL ATTAINMENT - Population 25 years and over - Associate's degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - High school graduate (includes equivalency),EDUCATIONAL ATTAINMENT - Population 25 years and over - High school graduate or higher,EDUCATIONAL ATTAINMENT - Population 25 years and over - Less than 9th grade,"EDUCATIONAL ATTAINMENT - Population 25 years and over - Some college, no degree",FERTILITY - Number of women 15 to 50 years old who had a birth in the past 12 months,"FERTILITY - Unmarried women (widowed, divorced, and never married)",Females 15 years and over,Females 15 years and over - Divorced,Females 15 years and over - Never married,"Females 15 years and over - Now married, except separated",Females 15 years and over - Separated,Females 15 years and over - Widowed,Foreign-born population - Naturalized U.S. citizen,Foreign-born population - Not a U.S. citizen,GRANDPARENTS - Number of grandparents living with own grandchildren under 18 years,GRANDPARENTS - Number of grandparents living with own grandchildren under 18 years - Grandparents responsible for grandchildren,Grandparents responsible for grandchildren - Years responsible for grandchildren - 1 or 2 years,Grandparents responsible for grandchildren - Years responsible for grandchildren - 3 or 4 years,Grandparents responsible for grandchildren - Years responsible for grandchildren - 5 or more years,Grandparents responsible for grandchildren - Years responsible for grandchildren - Less than 1 year,Households with one or more people 65 years and over,Households with one or more people under 18 years,LANGUAGE SPOKEN AT HOME - Asian and Pacific Islander languages,"LANGUAGE SPOKEN AT HOME - Asian and Pacific Islander languages - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - English only,LANGUAGE SPOKEN AT HOME - Language other than English,LANGUAGE SPOKEN AT HOME - Language other than English - Other languages,"LANGUAGE SPOKEN AT HOME - Language other than English - Other languages - Speak English less than ""very well""","LANGUAGE SPOKEN AT HOME - Language other than English - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - Other Indo-European languages,"LANGUAGE SPOKEN AT HOME - Other Indo-European languages - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - Population 5 years and over,LANGUAGE SPOKEN AT HOME - Spanish,Males 15 years and over,Males 15 years and over - Divorced,Males 15 years and over - Never married,"Males 15 years and over - Now married, except separated",Males 15 years and over - Separated,Males 15 years and over - Widowed,Number of grandparents responsible for own grandchildren under 18 years - Who are female,Number of grandparents responsible for own grandchildren under 18 years - Who are marrie

In [156]:
## PROCESS ALL 50 STATES SOCIAL FUNCTION
def process_social(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_social(data)
        data = clean_up_data_social(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfsocial_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctsocial_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfsocial_all, df_pctsocial_all


In [157]:
import pandas as pd

def process_social(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_social(data)
            data = clean_up_data_social(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # Optionally, print or log the rows causing the error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfsocial_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pctsocial_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfsocial_all, df_pctsocial_all, error_files

# Usage
file_state_list = file_state_list = [
 ('data/upper/ALLSLDU_DP02_AK.csv', 'AK'),('data/upper/ALLSLDU_DP02_AL.csv', 'AL'),('data/upper/ALLSLDU_DP02_AR.csv', 'AR'),
 ('data/upper/ALLSLDU_DP02_AZ.csv', 'AZ'),('data/upper/ALLSLDU_DP02_CA.csv', 'CA'),('data/upper/ALLSLDU_DP02_CO.csv', 'CO'),
 ('data/upper/ALLSLDU_DP02_CT.csv', 'CT'),('data/upper/ALLSLDU_DP02_DE.csv', 'DE'),('data/upper/ALLSLDU_DP02_FL.csv', 'FL'),
 ('data/upper/ALLSLDU_DP02_GA.csv', 'GA'),('data/upper/ALLSLDU_DP02_HI.csv', 'HI'),('data/upper/ALLSLDU_DP02_IA.csv', 'IA'),
 ('data/upper/ALLSLDU_DP02_ID.csv', 'ID'),('data/upper/ALLSLDU_DP02_IL.csv', 'IL'),('data/upper/ALLSLDU_DP02_IN.csv', 'IN'),
 ('data/upper/ALLSLDU_DP02_KS.csv', 'KS'),('data/upper/ALLSLDU_DP02_KY.csv', 'KY'),('data/upper/ALLSLDU_DP02_LA.csv', 'LA'),
 ('data/upper/ALLSLDU_DP02_MA.csv', 'MA'),('data/upper/ALLSLDU_DP02_MD.csv', 'MD'),('data/upper/ALLSLDU_DP02_ME.csv', 'ME'),
 ('data/upper/ALLSLDU_DP02_MI.csv', 'MI'),('data/upper/ALLSLDU_DP02_MN.csv', 'MN'),('data/upper/ALLSLDU_DP02_MO.csv', 'MO'),
 ('data/upper/ALLSLDU_DP02_MS.csv', 'MS'),('data/upper/ALLSLDU_DP02_MT.csv', 'MT'),('data/upper/ALLSLDU_DP02_NC.csv', 'NC'),
 ('data/upper/ALLSLDU_DP02_ND.csv', 'ND'),('data/upper/ALLSLDU_DP02_NE.csv', 'NE'),('data/upper/ALLSLDU_DP02_NH.csv', 'NH'),
 ('data/upper/ALLSLDU_DP02_NJ.csv', 'NJ'),('data/upper/ALLSLDU_DP02_NM.csv', 'NM'),('data/upper/ALLSLDU_DP02_NV.csv', 'NV'),
 ('data/upper/ALLSLDU_DP02_NY.csv', 'NY'),('data/upper/ALLSLDU_DP02_OH.csv', 'OH'),('data/upper/ALLSLDU_DP02_OK.csv', 'OK'),
 ('data/upper/ALLSLDU_DP02_OR.csv', 'OR'),('data/upper/ALLSLDU_DP02_PA.csv', 'PA'),('data/upper/ALLSLDU_DP02_RI.csv', 'RI'),
 ('data/upper/ALLSLDU_DP02_SC.csv', 'SC'),('data/upper/ALLSLDU_DP02_SD.csv', 'SD'),('data/upper/ALLSLDU_DP02_TN.csv', 'TN'),
 ('data/upper/ALLSLDU_DP02_TX.csv', 'TX'),('data/upper/ALLSLDU_DP02_UT.csv', 'UT'),('data/upper/ALLSLDU_DP02_VA.csv', 'VA'),
 ('data/upper/ALLSLDU_DP02_VT.csv', 'VT'),('data/upper/ALLSLDU_DP02_WA.csv', 'WA'),('data/upper/ALLSLDU_DP02_WI.csv', 'WI'),
 ('data/upper/ALLSLDU_DP02_WV.csv', 'WV'),('data/upper/ALLSLDU_DP02_WY.csv', 'WY')
]
df_prfsocial_all, df_pctsocial_all, error_files = process_social(file_state_list)

if error_files:
    print("Files with errors:", error_files)


In [158]:
df_prfsocial_all

,Formatted_District,GEOID,ANCESTRY - American,ANCESTRY - Arab,ANCESTRY - Czech,ANCESTRY - Danish,ANCESTRY - Dutch,ANCESTRY - English,ANCESTRY - French (except Basque),ANCESTRY - French Canadian,ANCESTRY - German,ANCESTRY - Greek,ANCESTRY - Hungarian,ANCESTRY - Irish,ANCESTRY - Italian,ANCESTRY - Lithuanian,ANCESTRY - Norwegian,ANCESTRY - Polish,ANCESTRY - Portuguese,ANCESTRY - Russian,ANCESTRY - Scotch-Irish,ANCESTRY - Scottish,ANCESTRY - Slovak,ANCESTRY - Subsaharan African,ANCESTRY - Swedish,ANCESTRY - Swiss,ANCESTRY - Ukrainian,ANCESTRY - Welsh,ANCESTRY - West Indian (excluding Hispanic origin groups),COMPUTERS AND INTERNET USE - Total households,COMPUTERS AND INTERNET USE - Total households - With a broadband Internet subscription,COMPUTERS AND INTERNET USE - Total households - With a computer,EDUCATIONAL ATTAINMENT - Population 25 years and over,"EDUCATIONAL ATTAINMENT - Population 25 years and over - 9th to 12th grade, no diploma",EDUCATIONAL ATTAINMENT - Population 25 years and over - Associate's degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,EDUCATIONAL ATTAINMENT - Population 25 years and over - High school graduate (includes equivalency),EDUCATIONAL ATTAINMENT - Population 25 years and over - High school graduate or higher,EDUCATIONAL ATTAINMENT - Population 25 years and over - Less than 9th grade,"EDUCATIONAL ATTAINMENT - Population 25 years and over - Some college, no degree",FERTILITY - Number of women 15 to 50 years old who had a birth in the past 12 months,"FERTILITY - Unmarried women (widowed, divorced, and never married)",Females 15 years and over,Females 15 years and over - Divorced,Females 15 years and over - Never married,"Females 15 years and over - Now married, except separated",Females 15 years and over - Separated,Females 15 years and over - Widowed,Foreign-born population - Naturalized U.S. citizen,Foreign-born population - Not a U.S. citizen,GRANDPARENTS - Number of grandparents living with own grandchildren under 18 years,GRANDPARENTS - Number of grandparents living with own grandchildren under 18 years - Grandparents responsible for grandchildren,Grandparents responsible for grandchildren - Years responsible for grandchildren - 1 or 2 years,Grandparents responsible for grandchildren - Years responsible for grandchildren - 3 or 4 years,Grandparents responsible for grandchildren - Years responsible for grandchildren - 5 or more years,Grandparents responsible for grandchildren - Years responsible for grandchildren - Less than 1 year,Households with one or more people 65 years and over,Households with one or more people under 18 years,LANGUAGE SPOKEN AT HOME - Asian and Pacific Islander languages,"LANGUAGE SPOKEN AT HOME - Asian and Pacific Islander languages - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - English only,LANGUAGE SPOKEN AT HOME - Language other than English,LANGUAGE SPOKEN AT HOME - Language other than English - Other languages,"LANGUAGE SPOKEN AT HOME - Language other than English - Other languages - Speak English less than ""very well""","LANGUAGE SPOKEN AT HOME - Language other than English - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - Other Indo-European languages,"LANGUAGE SPOKEN AT HOME - Other Indo-European languages - Speak English less than ""very well""",LANGUAGE SPOKEN AT HOME - Population 5 years and over,LANGUAGE SPOKEN AT HOME - Spanish,Males 15 years and over,Males 15 years and over - Divorced,Males 15 years and over - Never married,"Males 15 years and over - Now married, except separated",Males 15 years and over - Separated,Males 15 years and over - Widowed,Number of grandparents responsible for own grandchildren under 18 years - Who are female,Number of grandparents responsible for own grandchildren under 18 years - Who are marrie

In [159]:
# Assuming 'data' is your DataFrame
nan_counts = data.isna().sum()
print(nan_counts)


GEOID                 0
District              0
PROFLN                0
TITLE                 0
PRF_ESTIMATE          0
PCT_ESTIMATE          0
Formatted_District    0
dtype: int64


# Economic Characteristics

In [160]:
AL_econ_df = load_and_rename_data('data/upper/ALLSLDU_DP03_AL.csv')

In [161]:
def update_titles_econ(data):
   
    # Shorten two long titles--
    #INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)
    data.loc[data['PROFLN'] == '50.5', 'TITLE'] = 'INCOME AND BENEFITS' 
    #PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL
    data.loc[data['PROFLN'] == '118.5', 'TITLE'] = 'INCOME BELOW POVERTY LEVEL' 
   
    # now create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))

    ####### DICTIONARIES FOR PROFLN NUMBERS THAT NEED A DIRECT MAPPING#######
    
    # Dictionary "MISC FEMALE EMPLOYMENT"
    misc_female_employment_dict = {
        '15': '0.8-1-14-15',
        '16': '0.8-10-16',
        '17': '0.8-10-16-17'
    }
    # Dictionary "MISC INCOME AND BENEFITS" category
    misc_income_and_benefits_dict = {
        '65': '50.5-51-64-65',
        '66': '50.5-51-66',
        '67': '50.5-51-66-67',
        '68': '50.5-51-68',
        '69': '50.5-51-68-69',
        '70': '50.5-51-70',
        '71': '50.5-51-70-71',
        '72': '50.5-51-72',
        '73': '50.5-51-72-73',
        '74': '50.5-51-74',
    }
    
    # Dictionary HEALTH INSURANCE COVERAGE category
    health_insurance_coverage_dict = {
        '95': '94.5-95',
        '96': '94.5-95-96',
        '97': '94.5-95-97',
        '98': '94.5-95-98',
        '99': '94.5-95-99',
        '100': '94.5-100',
        '101': '94.5-100-101',
        '103': '102-103',
        '104': '94.5-104',
        '105': '94.5-104-105',
        '106': '94.5-104-106',
        '107': '94.5-104-107',
        '108': '94.5-104-108',
        '109': '94.5-109',
        '110': '94.5-109-110',
        '111': '94.5-109-111',
        '112': '94.5-109-112',
        '113': '94.5-109-113',
        '114': '94.5-114',
        '115': '94.5-114-115',
        '116': '94.5-114-116',
        '117': '94.5-114-117',
        '118': '94.5-114-118'
    }
    # Dictionary FAMILY POVERTY LEVEL
    family_poverty_level_dict = {
        '119': '118.5-119',
        '120': '118.5-119-120',
        '121': '118.5-119-121',
        '122': '118.5-122',
        '123': '118.5-122-123',
        '124': '118.5-122-124',
        '125': '118.5-125',
        '126': '118.5-125-126',
        '127': '118.5-125-127'
    }


    ###### PROFLN NUMBERS THAT FOLLOW A SIMPLE PATTERN OVER A LONG RANGE######
    # EMPLOYMENT STATUS category
    for i in range(1, 10):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-1' + ('' if i == 1 else '-' + str(i))
    # "FEMALE EMPLOYMENT STATUS" category
    for i in range(10, 15):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-10' + ('' if i == 10 else '-' + str(i))
    # "COMMUTING TO WORK" category
    for i in range(18, 26):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '17.5-' + str(i)
    # "OCCUPATION" category
    for i in range(26, 32):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '25.5-' + str(i)
    # "INDUSTRY" category
    for i in range(33, 46):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '31.5-' + str(i)
    #"CLASS OF WORKER" category
    for i in range(47, 51):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '45.5-' + str(i)
    # "INCOME AND BENEFITS" category
    for i in range(52, 65):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-51-' + str(i)
    # FAMILY INCOME AND BENEFITS category
    for i in range(75, 88):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-75' + ('' if i == 75 else '-' + str(i))
    # NON-FAMILY INCOME AND BENEFITS category
    for i in range(88, 95):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-' + str(i)
    # ALL PEOPLE POVERTY LEVEL category
    for i in range (128,138):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '118.5-128'+ ('' if i == 128 else '-' + str(i))
    
    
    ###### DIRECT MAPPING #####
    for old, new in misc_female_employment_dict.items(): #Misc Female Employment
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in misc_income_and_benefits_dict.items(): #Misc income and benefits
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in health_insurance_coverage_dict.items(): #Health Insurance Coverage
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in family_poverty_level_dict.items():     # Family poverty level
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    

    # Translate TITLE values using previously defined 'translate_profln' function
    data['PROFLN'] = data['PROFLN'].astype(str)
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))

    return data


In [162]:
AL_econ_df = update_titles_econ(AL_econ_df)
AL_econ_df[['PROFLN','TITLE']]

,PROFLN,TITLE
0,0,SELECTED ECONOMIC CHARACTERISTICS
1,0.5,nan
2,0.8,EMPLOYMENT STATUS
3,0.8-1,EMPLOYMENT STATUS - Population 16 years and over
4,0.8-1-2,EMPLOYMENT STATUS - Population 16 years and over - In labor force
...,...,...
5875,118.5-128-133,INCOME BELOW POVERTY LEVEL - All people - 18 years and over
5876,118.5-128-134,INCOME BELOW POVERTY LEVEL - All people - 18 to 64 years
5877,118.5-128-135,INCOME BELOW POVERTY LEVEL - All people - 65 years and over
5878,118.5-128-136,INCOME BELOW POVERTY LEVEL - All people - People in families


In [163]:
def clean_up_data_econ(data):
    
    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (and vice versa)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']
    data.loc[data['PRF_ESTIMATE'] == '(X)', 'PRF_ESTIMATE'] = data['PCT_ESTIMATE']
     # Replace '-' in PCT_ESTIMATE with 0 (inspection of data shows at or close to zero but not enough samples)
    data.loc[data['PCT_ESTIMATE'] == '-', 'PCT_ESTIMATE'] = '0'
    data.loc[data['PRF_ESTIMATE'] == '-', 'PRF_ESTIMATE'] = '0'
    
    # Drop unnecessary rows and rows with placeholders
    invalid_rows = (data['PRF_ESTIMATE'].isin(['N', '(X)']) | 
                data['PCT_ESTIMATE'].isin(['N', '(X)']) |
                data['PRF_ESTIMATE'].isna())
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['25.5-26', '32', '46',   # Duplicated 'Employed population 16+'
                        '0.8-1-8']               # Duplicated 'Civilian labor force'
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])


    # Convert to number - Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data


In [164]:
AL_econ_df = clean_up_data_econ(AL_econ_df)

In [165]:
format_districts(AL_econ_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
3,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1,EMPLOYMENT STATUS - Population 16 years and over,118470.0,118470.0,AL-Sen-01
4,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1-2,EMPLOYMENT STATUS - Population 16 years and over - In labor force,69215.0,58.4,AL-Sen-01
5,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1-3,EMPLOYMENT STATUS - Population 16 years and over - Civilian labor force,69103.0,58.3,AL-Sen-01
6,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1-4,EMPLOYMENT STATUS - Population 16 years and over - Employed,66408.0,56.1,AL-Sen-01
7,610U800US01001,"State Senate District 1 (2022), Alabama",0.8-1-5,EMPLOYMENT STATUS - Population 16 years and over - Unemployed,2695.0,2.3,AL-Sen-01
...,...,...,...,...,...,...,...
5875,610U800US01035,"State Senate District 35 (2022), Alabama",118.5-128-133,INCOME BELOW POVERTY LEVEL - All people - 18 years and over,11.7,11.7,AL-Sen-35
5876,610U800US01035,"State Senate District 35 (2022), Alabama",118.5-128-134,INCOME BELOW POVERTY LEVEL - All people - 18 to 64 years,12.9,12.9,AL-Sen-35
5877,610U800US01035,"State Senate District 35 (2022), Alabama",118.5-128-135,INCOME BELOW POVERTY LEVEL - All people - 65 years and over,7.6,7.6,AL-Sen-35
5878,610U800US01035,"State Senate District 35 (2022), Alabama",118.5-128-136,INCOME BELOW POVERTY LEVEL - All people - People in families,11.4,11.4,AL-Sen-35


In [166]:
AL_econ_prf, AL_econ_pct = reorient_dfs(AL_econ_df)
AL_econ_prf

,Formatted_District,GEOID,CLASS OF WORKER - Government workers,CLASS OF WORKER - Private wage and salary workers,CLASS OF WORKER - Self-employed in own not incorporated business workers,CLASS OF WORKER - Unpaid family workers,"COMMUTING TO WORK - Car, truck, or van -- carpooled","COMMUTING TO WORK - Car, truck, or van -- drove alone",COMMUTING TO WORK - Mean travel time to work (minutes),COMMUTING TO WORK - Other means,COMMUTING TO WORK - Public transportation (excluding taxicab),COMMUTING TO WORK - Walked,COMMUTING TO WORK - Worked from home,COMMUTING TO WORK - Workers 16 years and over,Civilian noninstitutionalized population 19 to 64 years,Civilian noninstitutionalized population 19 to 64 years - In labor force:,EMPLOYMENT STATUS - Females 16 years and over,EMPLOYMENT STATUS - Females 16 years and over - Civilian labor force,EMPLOYMENT STATUS - Females 16 years and over - Employed,EMPLOYMENT STATUS - Females 16 years and over - In labor force,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder 6 to 17 years,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder 6 to 17 years - All parents in family in labor force,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder under 6 years,EMPLOYMENT STATUS - Population 16 years and over,EMPLOYMENT STATUS - Population 16 years and over - Armed Forces,EMPLOYMENT STATUS - Population 16 years and over - Civilian labor force,EMPLOYMENT STATUS - Population 16 years and over - Employed,EMPLOYMENT STATUS - Population 16 years and over - In labor force,EMPLOYMENT STATUS - Population 16 years and over - Not in labor force,EMPLOYMENT STATUS - Population 16 years and over - Own children of the householder under 6 years - All parents in family in labor force,EMPLOYMENT STATUS - Population 16 years and over - Unemployed,EMPLOYMENT STATUS - Population 16 years and over - Unemployment Rate,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With private health insurance,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With public coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population under 19 years,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population under 19 years - No health insurance coverage,HEALTH INSURANCE COVERAGE - Employed:,HEALTH INSURANCE COVERAGE - Employed: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Employed: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Employed: - With private health insurance,HEALTH INSURANCE COVERAGE - Employed: - With public coverage,HEALTH INSURANCE COVERAGE - Not in labor force:,HEALTH INSURANCE COVERAGE - Not in labor force: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Not in labor force: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Not in labor force: - With private health insurance,HEALTH INSURANCE COVERAGE - Not in labor force: - With public coverage,HEALTH INSURANCE COVERAGE - Unemployed:,HEALTH INSURANCE COVERAGE - Unemployed: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Unemployed: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Unemployed: - With private health insurance,HEALTH INSURANCE COVERAGE - Unemployed: - With public coverage,INCOME AND BENEFITS - Families,"INCOME AND BENEFITS - Families - $10,000 to $14,999","INCOME AND BENEFITS - Families - $100,000 to $149,999","INCOME AND BENEFITS - Families - $15,000 to $24,999","INCOME AND BENEFITS - Families - $150,000 to $199,999","INCOME AND BENEFITS - Families - $200,000 or more","INCOME AND BENEFITS - Families - $25,000 to $34,999","INCOME AND BENEFITS - Families - $35,000 to $49,999","INCOME AN

In [167]:
AL_econ_pct.to_csv('data/converted_PCT_ALLSLDU_P03_AL.csv')

In [168]:
## PROCESS ALL 50 STATES HOUSING FUNCTION
def process_econ(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_econ(data)
        data = clean_up_data_econ(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfecon_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctecon_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfecon_all, df_pctecon_all


In [169]:
def process_econ(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_econ(data)
            data = clean_up_data_econ(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # Optionally, print or log the rows causing the error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfecon_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pctecon_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfecon_all, df_pctecon_all, error_files

# Usage
file_state_list = [
 ('data/upper/ALLSLDU_DP03_AK.csv', 'AK'),('data/upper/ALLSLDU_DP03_AL.csv', 'AL'),('data/upper/ALLSLDU_DP03_AR.csv', 'AR'),
 ('data/upper/ALLSLDU_DP03_AZ.csv', 'AZ'),('data/upper/ALLSLDU_DP03_CA.csv', 'CA'),('data/upper/ALLSLDU_DP03_CO.csv', 'CO'),
 ('data/upper/ALLSLDU_DP03_CT.csv', 'CT'),('data/upper/ALLSLDU_DP03_DE.csv', 'DE'),('data/upper/ALLSLDU_DP03_FL.csv', 'FL'),
 ('data/upper/ALLSLDU_DP03_GA.csv', 'GA'),('data/upper/ALLSLDU_DP03_HI.csv', 'HI'),('data/upper/ALLSLDU_DP03_IA.csv', 'IA'),
 ('data/upper/ALLSLDU_DP03_ID.csv', 'ID'),('data/upper/ALLSLDU_DP03_IL.csv', 'IL'),('data/upper/ALLSLDU_DP03_IN.csv', 'IN'),
 ('data/upper/ALLSLDU_DP03_KS.csv', 'KS'),('data/upper/ALLSLDU_DP03_KY.csv', 'KY'),('data/upper/ALLSLDU_DP03_LA.csv', 'LA'),
 ('data/upper/ALLSLDU_DP03_MA.csv', 'MA'),('data/upper/ALLSLDU_DP03_MD.csv', 'MD'),('data/upper/ALLSLDU_DP03_ME.csv', 'ME'),
 ('data/upper/ALLSLDU_DP03_MI.csv', 'MI'),('data/upper/ALLSLDU_DP03_MN.csv', 'MN'),('data/upper/ALLSLDU_DP03_MO.csv', 'MO'),
 ('data/upper/ALLSLDU_DP03_MS.csv', 'MS'),('data/upper/ALLSLDU_DP03_MT.csv', 'MT'),('data/upper/ALLSLDU_DP03_NC.csv', 'NC'),
 ('data/upper/ALLSLDU_DP03_ND.csv', 'ND'),('data/upper/ALLSLDU_DP03_NE.csv', 'NE'),('data/upper/ALLSLDU_DP03_NH.csv', 'NH'),
 ('data/upper/ALLSLDU_DP03_NJ.csv', 'NJ'),('data/upper/ALLSLDU_DP03_NM.csv', 'NM'),('data/upper/ALLSLDU_DP03_NV.csv', 'NV'),
 ('data/upper/ALLSLDU_DP03_NY.csv', 'NY'),('data/upper/ALLSLDU_DP03_OH.csv', 'OH'),('data/upper/ALLSLDU_DP03_OK.csv', 'OK'),
 ('data/upper/ALLSLDU_DP03_OR.csv', 'OR'),('data/upper/ALLSLDU_DP03_PA.csv', 'PA'),('data/upper/ALLSLDU_DP03_RI.csv', 'RI'),
 ('data/upper/ALLSLDU_DP03_SC.csv', 'SC'),('data/upper/ALLSLDU_DP03_SD.csv', 'SD'),('data/upper/ALLSLDU_DP03_TN.csv', 'TN'),
 ('data/upper/ALLSLDU_DP03_TX.csv', 'TX'),('data/upper/ALLSLDU_DP03_UT.csv', 'UT'),('data/upper/ALLSLDU_DP03_VA.csv', 'VA'),
 ('data/upper/ALLSLDU_DP03_VT.csv', 'VT'),('data/upper/ALLSLDU_DP03_WA.csv', 'WA'),('data/upper/ALLSLDU_DP03_WI.csv', 'WI'),
 ('data/upper/ALLSLDU_DP03_WV.csv', 'WV'),('data/upper/ALLSLDU_DP03_WY.csv', 'WY')
]

df_prfecon_all, df_pctecon_all, error_files = process_econ(file_state_list)

if error_files:
    print("Files with errors:", error_files)


In [170]:
df_pctecon_all

,Formatted_District,GEOID,CLASS OF WORKER - Government workers,CLASS OF WORKER - Private wage and salary workers,CLASS OF WORKER - Self-employed in own not incorporated business workers,CLASS OF WORKER - Unpaid family workers,"COMMUTING TO WORK - Car, truck, or van -- carpooled","COMMUTING TO WORK - Car, truck, or van -- drove alone",COMMUTING TO WORK - Mean travel time to work (minutes),COMMUTING TO WORK - Other means,COMMUTING TO WORK - Public transportation (excluding taxicab),COMMUTING TO WORK - Walked,COMMUTING TO WORK - Worked from home,COMMUTING TO WORK - Workers 16 years and over,Civilian noninstitutionalized population 19 to 64 years,Civilian noninstitutionalized population 19 to 64 years - In labor force:,EMPLOYMENT STATUS - Females 16 years and over,EMPLOYMENT STATUS - Females 16 years and over - Civilian labor force,EMPLOYMENT STATUS - Females 16 years and over - Employed,EMPLOYMENT STATUS - Females 16 years and over - In labor force,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder 6 to 17 years,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder 6 to 17 years - All parents in family in labor force,EMPLOYMENT STATUS - Females 16 years and over - Own children of the householder under 6 years,EMPLOYMENT STATUS - Population 16 years and over,EMPLOYMENT STATUS - Population 16 years and over - Armed Forces,EMPLOYMENT STATUS - Population 16 years and over - Civilian labor force,EMPLOYMENT STATUS - Population 16 years and over - Employed,EMPLOYMENT STATUS - Population 16 years and over - In labor force,EMPLOYMENT STATUS - Population 16 years and over - Not in labor force,EMPLOYMENT STATUS - Population 16 years and over - Own children of the householder under 6 years - All parents in family in labor force,EMPLOYMENT STATUS - Population 16 years and over - Unemployed,EMPLOYMENT STATUS - Population 16 years and over - Unemployment Rate,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With private health insurance,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With public coverage,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population under 19 years,HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population under 19 years - No health insurance coverage,HEALTH INSURANCE COVERAGE - Employed:,HEALTH INSURANCE COVERAGE - Employed: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Employed: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Employed: - With private health insurance,HEALTH INSURANCE COVERAGE - Employed: - With public coverage,HEALTH INSURANCE COVERAGE - Not in labor force:,HEALTH INSURANCE COVERAGE - Not in labor force: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Not in labor force: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Not in labor force: - With private health insurance,HEALTH INSURANCE COVERAGE - Not in labor force: - With public coverage,HEALTH INSURANCE COVERAGE - Unemployed:,HEALTH INSURANCE COVERAGE - Unemployed: - No health insurance coverage,HEALTH INSURANCE COVERAGE - Unemployed: - With health insurance coverage,HEALTH INSURANCE COVERAGE - Unemployed: - With private health insurance,HEALTH INSURANCE COVERAGE - Unemployed: - With public coverage,INCOME AND BENEFITS - Families,"INCOME AND BENEFITS - Families - $10,000 to $14,999","INCOME AND BENEFITS - Families - $100,000 to $149,999","INCOME AND BENEFITS - Families - $15,000 to $24,999","INCOME AND BENEFITS - Families - $150,000 to $199,999","INCOME AND BENEFITS - Families - $200,000 or more","INCOME AND BENEFITS - Families - $25,000 to $34,999","INCOME AND BENEFITS - Families - $35,000 to $49,999","INCOME AN

# Merging all datasets

In [171]:
#merge all the PERCENT dataframes

df_pct_merged = pd.merge(df_pctdemo_all, df_pcthousing_all, on=['Formatted_District', 'GEOID'])
df_pct_merged = pd.merge(df_pct_merged, df_pctsocial_all, on=['Formatted_District', 'GEOID'])
df_pct_merged = pd.merge(df_pct_merged, df_pctecon_all, on=['Formatted_District', 'GEOID'])
# check shape to ensure we maintained all 435 districts
print(df_pct_merged.head())
print(df_pct_merged.shape)

                       Formatted_District           GEOID  \
0  State Senate District A (2022), Alaska  610U800US0200A   
1  State Senate District B (2022), Alaska  610U800US0200B   
2  State Senate District C (2022), Alaska  610U800US0200C   
3  State Senate District D (2022), Alaska  610U800US0200D   
4  State Senate District E (2022), Alaska  610U800US0200E   

   CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population  \
0                                                           27426.0   
1                                                           27677.0   
2                                                           26984.0   
3                                                           27906.0   
4                                                           26259.0   

   CITIZEN, VOTING AGE POPULATION - Female  \
0                                     46.6   
1                                     49.1   
2                                     46.0   
3                     

In [172]:
#merge all the RAW NUMBER (PRF) dataframes

df_prf_merged = pd.merge(df_prfdemo_all, df_prfhousing_all, on=['Formatted_District', 'GEOID'])
df_prf_merged = pd.merge(df_prf_merged, df_prfsocial_all, on=['Formatted_District', 'GEOID'])
df_prf_merged = pd.merge(df_prf_merged, df_prfecon_all, on=['Formatted_District', 'GEOID'])
# check shape to ensure we maintained all 435 districts
print(df_prf_merged.head())
print(df_prf_merged.shape)

                       Formatted_District           GEOID  \
0  State Senate District A (2022), Alaska  610U800US0200A   
1  State Senate District B (2022), Alaska  610U800US0200B   
2  State Senate District C (2022), Alaska  610U800US0200C   
3  State Senate District D (2022), Alaska  610U800US0200D   
4  State Senate District E (2022), Alaska  610U800US0200E   

   CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population  \
0                                                           27426.0   
1                                                           27677.0   
2                                                           26984.0   
3                                                           27906.0   
4                                                           26259.0   

   CITIZEN, VOTING AGE POPULATION - Female  \
0                                  12789.0   
1                                  13577.0   
2                                  12419.0   
3                     

In [173]:
df_pct_merged.to_csv('data/ACS_ALLSLDU_Percentages.csv')
df_prf_merged.to_csv('data/ACS_ALLSLDU_Population.csv')

# Tidying Up

In [174]:
# duplicated columns between datsets
columns_to_delete = [
    'HOUSING OCCUPANCY - Total housing units',
    'PLACE OF BIRTH - Total population',
    'EMPLOYMENT STATUS - Population 16 years and over',
    'HOUSING OCCUPANCY - Occupied housing units',
    'COMPUTERS AND INTERNET USE - Total households',
    'Population in households - Householder',
    'Population born outside the United States - Native',
    'Total Civilian Noninstitutionalized Population'
]


# deleting the columns
df_pct_merged = df_pct_merged.drop(columns=columns_to_delete)
df_prf_merged = df_prf_merged.drop(columns=columns_to_delete)


In [175]:
# Identify numeric columns
numeric_cols_prf = df_prf_merged.select_dtypes(include=np.number).columns.tolist()
numeric_cols_pct = df_pct_merged.select_dtypes(include=np.number).columns.tolist()

# Replace NaNs in numeric columns with their median
df_prf_merged.loc[:, numeric_cols_prf] = df_prf_merged.loc[:, numeric_cols_prf].fillna(df_prf_merged[numeric_cols_prf].median())
df_pct_merged.loc[:, numeric_cols_pct] = df_pct_merged.loc[:, numeric_cols_pct].fillna(df_pct_merged[numeric_cols_pct].median())
    

In [176]:


# Find columns with more than 100 NaNs
cols_to_drop_prf = df_prf_merged.columns[df_prf_merged.isnull().sum() > 100].tolist()
cols_to_drop_pct = df_pct_merged.columns[df_pct_merged.isnull().sum() > 100].tolist()

# # Exclude 'Other Votes' from the list
cols_to_drop_prf = [col for col in cols_to_drop_prf]
cols_to_drop_pct = [col for col in cols_to_drop_pct]

# Drop these columns
df_prf_merged.drop(columns=cols_to_drop_prf, inplace=True)
df_pct_merged.drop(columns=cols_to_drop_pct, inplace=True)

# For 'RACE - ...' and 'LANGUAGE SPOKEN AT HOME...' categories, replace NaNs with 0
race_language_cols_prf = [col for col in df_prf_merged if 'RACE -' in col or 'LANGUAGE SPOKEN AT HOME' in col]
race_language_cols_pct = [col for col in df_pct_merged if 'RACE -' in col or 'LANGUAGE SPOKEN AT HOME' in col]

df_prf_merged[race_language_cols_prf] = df_prf_merged[race_language_cols_prf].fillna(0)
df_pct_merged[race_language_cols_pct] = df_pct_merged[race_language_cols_pct].fillna(0)

# For the remaining numeric columns, replace NaNs with the column's median
df_prf_merged.loc[:, numeric_cols_prf] = df_prf_merged.loc[:, numeric_cols_prf].fillna(df_prf_merged[numeric_cols_prf].median())
df_pct_merged.loc[:, numeric_cols_pct] = df_pct_merged.loc[:, numeric_cols_pct].fillna(df_pct_merged[numeric_cols_pct].median())



In [177]:
df_pct_merged.shape

(1942, 453)

# Train PVI Model on Congressional Districts where we know the PVI scores

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [179]:
# Load datasets
pct_df = pd.read_csv('data/Full_Dataset_PCT.csv')  
prf_df = pd.read_csv('data/Full_Dataset_PRF.csv')  

# Define target variable
y = pct_df['PVI_2023']

# Define feature variables
X = pct_df.iloc[:, 3:454]  # first two columns are titles of districts, last columns are potential targets

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [180]:
# Retrain the model with the best metrics on our data
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Ridge(alpha=1))
])

# Train the model on the training data
best_model_pvi = pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model_pvi.predict(X_test)

# Calculate the R² score, RMSE, and MAE to confirm it's properly fit
r2_test = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

# Display the metrics
print(f"R² on Test Data: {r2_test:.4f}")
print(f"RMSE on Test Data: {rmse:.4f}")
print(f"MAE on Test Data: {mae:.4f}")


R² on Test Data: 0.9122
RMSE on Test Data: 4.3654
MAE on Test Data: 3.2661


In [181]:
# Assuming df_state_leg is your state legislative district dataframe and df_uscd is your US Congressional Districts dataframe

# Define the columns from USCD that are used as features in the model (excluding target and other non-relevant columns)
feature_columns = [col for col in pct_df.columns if col not in {'Unnamed: 0', 'Representative', 'PVI_2023', 'Rep', 'Status', 'Dem Votes', 'Rep Votes', 'Other Votes', 'Dem %', 'Rep %', 'Other %', '2022 Margin', 'Swing from 2020 presidential', 'Percent of 2020 turnout', '2022 State PVI', '2020 Biden %', '2020 Trump %', '2016 Clinton %', '2016 Trump %', 'National 2020 Pres Popular %', 'National 2022 House Popular %', '2020 State Diff', '2016 State Diff'}]

# Filter the state legislative districts dataframe to only include the feature columns
df_state_leg_filtered = df_pct_merged[feature_columns]

# Check if there are any missing columns in the state legislative districts dataframe
missing_columns = [col for col in feature_columns if col not in df_pct_merged.columns]
if missing_columns:
    print("Missing columns in state legislative districts dataset:", missing_columns)
else:
    print("All feature columns are present.")


All feature columns are present.


In [182]:
# Assume that the first two columns to be excluded are 'Formatted_District' and 'GEOID',
# and you need to take a specific range of columns based on how the features were arranged in the original dataset

# Using .iloc to select columns by index
X_state_leg = df_pct_merged.iloc[:, 2:453]  # Adjust indices to match feature columns; starting from index 2 to exclude the first two

# Use the trained model to predict PVI
df_pct_merged['PVI'] = best_model_pvi.predict(X_state_leg)

# Output the first few rows to verify the new PVI column
print(df_pct_merged[['Formatted_District', 'GEOID', 'PVI']].head())

# If you need to save the updated DataFrame:
df_pct_merged.to_csv('State_Legislative_Districts_with_PVI.csv', index=False)
print("PVI scores predicted and saved successfully.")


                       Formatted_District           GEOID        PVI
0  State Senate District A (2022), Alaska  610U800US0200A -15.935082
1  State Senate District B (2022), Alaska  610U800US0200B  -0.063119
2  State Senate District C (2022), Alaska  610U800US0200C  -9.441946
3  State Senate District D (2022), Alaska  610U800US0200D -28.885338
4  State Senate District E (2022), Alaska  610U800US0200E  -1.426942
PVI scores predicted and saved successfully.


In [183]:
df_pct_merged.to_csv('data/ACS_ALLSLDU_PVI.csv')